In [1]:
from helpers import load_bibliography, read, get_vals, remove_curlies, authors2key, same_id, check_entries, find_duplicates, check_key_suffixes, generate_correct_pages, format_journal_name, reformat_author, journal_key, publisher_key, address_codes, address_key, polish_database

In [2]:
bd = load_bibliography('../memlab.bib')
#bd = load_bibliography('cleaned.bib')

In [3]:
#template = load_bibliography('https://raw.githubusercontent.com/ContextLab/CDL-bibliography/master/memlab.bib')

In [4]:
bib_keys = list(bd.entries_dict.keys())
#template_keys = list(template.keys())

In [5]:
ids = get_vals(bd, 'ID')
authors = get_vals(bd, 'author')
years = get_vals(bd, 'year')
titles = get_vals(bd, 'title', proc=remove_curlies)
pages = get_vals(bd, 'pages')
journals = get_vals(bd, 'journal')
book_titles = get_vals(bd, 'booktitle')
publishers = get_vals(bd, 'publisher')
editors = get_vals(bd, 'editor')
addresses = get_vals(bd, 'address')

Check for duplicate keys

In [6]:
duplicate_keys, redundant_keys = find_duplicates(ids, authors, titles)

Checking for duplicated keys and entries...
No keys with multiple entries were found.
No entries with duplicated authors/titles were found.


In [7]:
fix_dict = {}
fix_dict['ID'] = check_entries('ID', bd, [authors2key(a, y) for a, y in zip(authors, years)], same=same_id);

running check: ID...
no IDs to fix!


Check for correct key suffixes

In [8]:
target_keys = check_key_suffixes(bd)
fix_dict['ID'].extend(check_entries('ID', bd, target_keys))

running check: ID...
3 errors detected:
LohnEtal11b: 	ID "LohnEtal11b" should be "LohnEtal11"
NelsEtal98a: 	ID "NelsEtal98a" should be "NelsEtal98"
Nair92b: 	ID "Nair92b" should be "Nair92"


Check page numbering

In [9]:
target_pages, unfixable = generate_correct_pages(bd)

In [10]:
if len(unfixable) > 0:
    print(f'The following page numbers are ambiguous or incorrect: \n')
    print('\n'.join([f'{i}: {p}' for i, p in unfixable]))
else:
    print('No ambiguous page numbers were found.')

No ambiguous page numbers were found.


In [11]:
fix_dict['pages'] = check_entries('pages', bd, target_pages)

running check: pages...
2441 errors detected:
MantEtal07: 	pages "13170-13175" should be "13170--13175"
Dobr70: 	pages "458-486" should be "458--486"
ByliEtal15: 	pages "165-178" should be "165--178"
vanKEtal13: 	pages "2352-2359" should be "2352--2359"
SereBoyn07: 	pages "301-312" should be "301--312"
LiuManc11: 	pages "26-33" should be "26--33"
Leml75: 	pages "235 -- 238" should be "235--238"
DroiWear15: 	pages "77 -- 82" should be "77--82"
Wear15: 	pages "165 -- 171" should be "165--171"
Howa18: 	pages "124-136" should be "124--136"
TigaEtal16: 	pages "5663-5671" should be "5663--5671"
TompEtal15: 	pages "7326-7331" should be "7326--7331"
NazaReza13: 	pages "54-61" should be "54--61"
DiazEtal19: 	pages "1-12" should be "1--12"
KitaEtal17: 	pages "73-78" should be "73--78"
TambDava13: 	pages "19591-19596" should be "19591--19596"
TambEtal10: 	pages "280-290" should be "280--290"
PlonEtal00: 	pages "1211-1216" should be "1211--1216"
HumpEtal07: 	pages "924-932" should be "924--932"
Oc

ShauEtal72: 	pages "1-12" should be "1--12"
TanEtal12: 	pages "1173-1183" should be "1173--1183"
TeppEtal95: 	pages "3092-3103" should be "3092--3103"
SohaHass98b: 	pages "171-93" should be "171--193"
SinkEtal95: 	pages "99-104" should be "99--104"
HerrMeck01: 	pages "593-608" should be "593--608"
BasaEtal01: 	pages "241-248" should be "241--248"
BaueEtal07: 	pages "9369---9379" should be "9369--9379"
KopeEtal00: 	pages "1867-1872" should be "1867--1872"
RaghEtal01: 	pages "3175-3183" should be "3175--3183"
SchiEtal99: 	pages "2182-2196" should be "2182--2196"
FrinEtal06a: 	pages "417-21" should be "417--421"
Schu55: 	pages "267-272" should be "267--272"
MitcRanc80: 	pages "49-66" should be "49--66"
Mart75: 	pages "150-153" should be "150--153"
WixtEbbe97: 	pages "731-739" should be "731--739"
LohmEtal08: 	pages "19096-19101" should be "19096--19101"
Katz36: 	pages "234-240" should be "234--240"
Kroh35: 	pages "150-190" should be "150--190"
HumpEtal89a: 	pages "36-67" should be "36--67

BoldEtal04: 	pages "353-361" should be "353--361"
NelsEtal97: 	pages "785-796" should be "785--796"
DawEtal02: 	pages "603-616" should be "603--616"
GradEtal09: 	pages "354-359" should be "354--359"
BlanEtal08b: 	pages "41-56" should be "41--56"
BassEtal10: 	pages "1344-1352" should be "1344--1352"
Wall07: 	pages "31-56" should be "31--56"
LeeEste77: 	pages "395-418" should be "395--418"
RaaiShif81b: 	pages "403-415" should be "403--415"
Rund78: 	pages "91-101" should be "91--101"
John72a: 	pages "125-159" should be "125--159"
ReitRuet80: 	pages "554-581" should be "554--581"
StusEtal94: 	pages "355--355" should be "355"
John72b: 	pages "300-307" should be "300--307"
TlauWils96: 	pages "647-664" should be "647--664"
RiekBayl00: 	pages "181-186" should be "181--186"
Free06b: 	pages "572-589" should be "572--589"
GlanGree07: 	pages "365-371" should be "365--371"
SaneDono93: 	pages "4470 - 4474" should be "4470--4474"
SeagEtal02: 	pages "1616--20" should be "1616--1620"
TallEtal97: 	pages

Murd61: 	pages "618-25" should be "618--625"
BlasEtal96: 	pages "161-164" should be "161--164"
KossEtal99: 	pages "167-170" should be "167--170"
RoedEtal98: 	pages "230-241" should be "230--241"
DrewMurd80: 	pages "319-332" should be "319--332"
PetePete57: 	pages "102-105" should be "102--105"
Wayn96: 	pages "523-533" should be "523--533"
Dona96: 	pages "523-533" should be "523--533"
Heft79: 	pages "172-185" should be "172--185"
Gree99: 	pages "309-312" should be "309--312"
LeeEtal05b: 	pages "145--53" should be "145--153"
DeLoMcDa96: 	pages "1136-1146" should be "1136--1146"
FellAxma11: 	pages "105-118" should be "105--118"
SohnEtal00: 	pages "13448-13453" should be "13448--13453"
Rock57: 	pages "186-193" should be "186--193"
PollGero68: 	pages "303-313" should be "303--313"
SlamBarl79: 	pages "617-627" should be "617--627"
LawsBlan93: 	pages "132-144" should be "132--144"
Cowa92: 	pages "1-17" should be "1--17"
deFoEtal01: 	pages "1803-1806" should be "1803--1806"
RypmDEsp99: 	pages 

Check journal names

In [12]:
fix_dict['journal'] = check_entries('journal', bd, [format_journal_name(j) for j in journals])

running check: journal...
1977 errors detected:
BeatEtal16: 	journal "{Trends in Cognitive Sciences}" should be "Trends in Cognitive Sciences"
TianEtal20: 	journal "Nature {N}euroscience" should be "Nature Neuroscience"
ArzyScha19: 	journal "{Trends in Cognitive Sciences}" should be "Trends in Cognitive Sciences"
GautEtal18: 	journal "{Cerebral Cortex}" should be "Cerebral Cortex"
KnigEich13: 	journal "Nature {N}euroscience" should be "Nature Neuroscience"
GautEtal12: 	journal "{The Journal of Neuroscience}" should be "The Journal of Neuroscience"
FentEtal08: 	journal "{The Journal of Neuroscience}" should be "The Journal of Neuroscience"
MikoEtal13b: 	journal "Proceedings of the NAACL-HLT" should be "Proceedings of the National Association for Computational Linguistics"
DeerEtal90: 	journal "Journal of the American Society for Information Science" should be "Journal of the {American} Society for Information Science"
Schr26: 	journal "The Physical Review" should be "Physical Review"
Sc

KaisLutz05: 	journal "Neuroreport" should be "{NeuroReport}"
CornEtal08: 	journal "Journal of Neuroscience" should be "The Journal of Neuroscience"
HalgEtal78a: 	journal "Electroencephalograph. Clin. Neurophysiol." should be "Electroencephalography and Clinical Neurophysiology"
BurkEtal14: 	journal "{N}euro{I}mage" should be "{NeuroImage}"
SedeEtal11: 	journal "Psychonomic Bulletin \& Review" should be "Psychonomic Bulletin and Review"
MillEtal12b: 	journal "PLoS Computational Biology" should be "{PLoS} Computational Biology"
RizzEtal06: 	journal "Neuro{I}mage" should be "{NeuroImage}"
BodiEtal05: 	journal "Brain research bulletin" should be "Brain Research Bulletin"
SummEtal11: 	journal "Front Hum Neurosci" should be "Frontiers in Human Neuroscience"
WangSpel02: 	journal "Trends Cogn Sci" should be "Trends in Cognitive Sciences"
CaplEtal03: 	journal "Journal of Neuroscience" should be "The Journal of Neuroscience"
Colt83: 	journal "Philos Trans R Soc Lond B Biol Sci" should be "Philos

HuanEtal04: 	journal "Memory \& Cognition" should be "Memory and Cognition"
WigEtal09: 	journal "J Neurophysiol" should be "Journal of Neurophysiology"
FrieEtal12: 	journal "Neuroreport" should be "{NeuroReport}"
BuchDEsp09: 	journal "Cereb Cortex" should be "Cerebral Cortex"
GrubEtal06: 	journal "Eur J Neurosci" should be "European Journal of Neuroscience"
JameGaut06: 	journal "Hum Brain Mapp" should be "Human Brain Mapping"
HowaEtal09: 	journal "Psychonomic Bulletin \& Review" should be "Psychonomic Bulletin and Review"
ShapEtal06: 	journal "Current opinion in neurobiology" should be "Current Opinion in Neurobiology"
BotvPlau02: 	journal "Psychol Res" should be "Psychological Research"
Hint11: 	journal "Perspectives on Psychological Perspectives on Psychological Science" should be "Perspectives on Psychological Science"
RizzEtal03: 	journal "Proceedings of the National Academy of Sciences, USA" should be "Proceedings of the National Academy of Sciences, {USA}"
HutcYaro00: 	journal "T

Check book titles

In [13]:
fix_dict['booktitle'] = check_entries('booktitle', bd, [format_journal_name(b) for b in book_titles])

running check: booktitle...
180 errors detected:
PennEtal14: 	booktitle "Proceedings of the 2014 Conference on Empirical Methods in Natural Language Processing" should be "Proceedings of the Conference on Empirical Methods in Natural Language Processing"
IyyeEtal15: 	booktitle "Proceedings of the 53rd Annual Meeting of the Association for Computational Linguistics and the 7th International Joint Conference on Natural Language Processing" should be "Proceedings of the Association for Computational Linguistics"
ViswEtal17: 	booktitle "31st Conference on Neural Information Processing Systems" should be "Advances in Neural Information Processing Systems"
SzpuTulv11: 	booktitle "Predictions in the brain: using our past to generate a future" should be "Predictions in the Brain: Using Our Past to Generate {A} Future"
Tulv02b: 	booktitle "Principles of frontal lobe function" should be "Principles of Frontal Lobe Function"
BobrNorm75: 	booktitle "Representation and understanding" should be "Rep

Check publishers

In [14]:
fix_dict['publisher'] = check_entries('publisher', bd, [format_journal_name(p, key=publisher_key) for p in publishers])

running check: publisher...
811 errors detected:
SzpuTulv11: 	publisher "Oxford University Press" should be "{Oxford} {University} Press"
Tulv02b: 	publisher "Oxford University Press" should be "{Oxford} {University} Press"
Mann20: 	publisher "Oxford University Press" should be "{Oxford} {University} Press"
Nune95: 	publisher "Oxford University Press" should be "{Oxford} {University} Press"
Rund71: 	publisher "American Psychological Association" should be "{American} Psychological Association"
PescEtal09: 	publisher "Elsevier Ltd" should be "Elsevier"
GajeFalk15: 	publisher "Elsevier Inc." should be "Elsevier"
ScudEtal14: 	publisher "Elsevier Inc." should be "Elsevier"
SkriEtal14: 	publisher "Elsevier Inc." should be "Elsevier"
BuddEtal10: 	publisher "Elsevier Ltd" should be "Elsevier"
SaadEtal14b: 	publisher "Elsevier Inc." should be "Elsevier"
NiemEtal13: 	publisher "Elsevier Ltd" should be "Elsevier"
McMoEtal11: 	publisher "Elsevier Inc." should be "Elsevier"
CookBeav13: 	publisher 

BravEtal07: 	publisher "Oxford University Press" should be "{Oxford} {University} Press"
KaneEtal07: 	publisher "Oxford University Press" should be "{Oxford} {University} Press"
CowaEtal04: 	publisher "Oxford Univ Press" should be "{Oxford} {University} Press"
NeweEtal01: 	publisher "John Wiley \& Sons" should be "Wiley"
WitmEtal96: 	publisher "London; San Diego: Academic Press, c1994-" should be "Academic Press"
Marr82: 	publisher "Henry Holt and Co., Inc. New York, NY, USA" should be "Henry Holt and Company"
VidaEtal06: 	publisher "MIT Press" should be "{MIT} Press"
Grah89: 	publisher "Oxford University Press" should be "{Oxford} {University} Press"
GrosEtal72: 	publisher "Am Physiological Soc" should be "{American} Physiological Society"
OlsoEtal06: 	publisher "MIT Press" should be "{MIT} Press"
BerrSwai89: 	publisher "American Psychological Association (PsycARTICLES)" should be "{American} Psychological Association"
FietEtal08: 	publisher "Soc Neuroscience" should be "Society for N

Check author names

In [15]:
fix_dict['author'] = check_entries('author', bd, [reformat_author(a) for a in authors])

running check: author...
5206 errors detected:
Gros88: 	author "Grossberg, S." should be "S Grossberg"
Fris00: 	author "KJ Friston" should be "K J Friston"
MedfCrit10: 	author "Medford, N. and Critchley, H. D." should be "N Medford and H D Critchley"
KellEtal07b: 	author "Kelly, S. and Lloyd, D. and Nurmikko, T. and Roberts, N." should be "S Kelly and D Lloyd and T Nurmikko and N Roberts"
JackWool18: 	author "Jackson, J. B. and Woolgar, A." should be "J B Jackson and A Woolgar"
CohnRang17: 	author "Cohn-Sheely, B I and Ranganath, C" should be "B I Cohn-Sheely and C Ranganath"
MantEtal07: 	author "Mantini, D. and Perrucci, M. G. and Del Gratta, C. and Romani, G. L. and Corbetta, M." should be "D Mantini and M G Perrucci and C {Del Gratta} and G L Romani and M Corbetta"
Dobr70: 	author "Dobrushin, R. L." should be "R L Dobrushin"
RamdEtal17: 	author "Ramdas, A. and Trillos, N. and Cuturi, M." should be "A Ramdas and N Trillos and M Cuturi"
ByliEtal15: 	author "Bylinskii, Z and Isola, P a

MarcEtal07: 	author "Marceglia, S. and Rossi, L. and Foffani, G. and Bianchi, A.M. and Cerutti, S. and Priori, A." should be "S Marceglia and L Rossi and G Foffani and A M Bianchi and S Cerutti and A Priori"
MiddStri00: 	author "Middleton, F. A. and Strick, P. L." should be "F A Middleton and P L Strick"
HeysEtal13: 	author "Heys, James G and MacLeod, Katrina M and Moss, Cynthia F and Hasselmo, Michael E" should be "James G Heys and Katrina M MacLeod and Cynthia F Moss and Michael E Hasselmo"
KassRaft95: 	author "Kass, R. E. and Raftery, A. E." should be "R E Kass and A E Raftery"
RangEtal06: 	author "Ranganathan, A. and Menegatti, E. and Dellaert, F." should be "A Ranganathan and E Menegatti and F Dellaert"
AharEtal01: 	author "Aharon, I. and Etcoff, N. and Ariely, D. and Chabris, C. F. and O'Connor, E. and Breiter, H. C." should be "I Aharon and N Etcoff and D Ariely and C F Chabris and E O'Connor and H C Breiter"
JacoEtal89: 	author "Jacoby, L. L. and Woloshyn, V. and Kelley, C." sh

RayMaun11a: 	author "Ray, S. and Maunsell, J. H. R." should be "S Ray and J H R Maunsell"
MurpEtal00: 	author "Murphy, M.H. and Nevill, A.M. and Hardman, A.E. and others" should be "M H Murphy and A M Nevill and A E Hardman and others"
DrucAgui09: 	author "Drucker, Daniel M and Aguirre, Geoffrey K" should be "Daniel M Drucker and Geoffrey K Aguirre"
SharEtal09: 	author "Sharott, A. and Moll, C. K. E. and Engler, G. and Denker, M. and Grun, S. and Engel, A. K." should be "A Sharott and C K E Moll and G Engler and M Denker and S Grun and A K Engel"
PasuMill05: 	author "Pasupathy, A. and Miller, E. K." should be "A Pasupathy and E K Miller"
EckhEtal04: 	author "Eckhorn, R. and Gail, A. and Bruns, A. and Gabriel, A. and Al-Shaikhli, B. and Saam, M." should be "R Eckhorn and A Gail and A Bruns and A Gabriel and B Al-Shaikhli and M Saam"
HumpEtal89b: 	author "Humphreys, M. S. and Bain, J. D. and Pike, R." should be "M S Humphreys and J D Bain and R Pike"
StepEtal05: 	author "Stephen, J. M. a

JutrEtal09: 	author "Jutras, M.J. and Fries, P. and Buffalo, E.A." should be "M J Jutras and P Fries and E A Buffalo"
VincEtal10: 	author "Vinck, M. and Lima, B. and Womelsdorf, T. and Oostenveld, R. and Singer, W. and Neuenschwander, S. and Fries, P." should be "M Vinck and B Lima and T Womelsdorf and R Oostenveld and W Singer and S Neuenschwander and P Fries"
RaghEtal01: 	author "Raghavachari, S. and Rizzuto, D. S. and Caplan, J. B. and Kirschen, M. P. and Bourgeois, B. and Madsen, J. R. and Kahana, M. J. and Lisman, J. E." should be "S Raghavachari and D S Rizzuto and J B Caplan and M P Kirschen and B Bourgeois and J R Madsen and M J Kahana and J E Lisman"
SchiEtal99: 	author "Schiff, N. D. and Purpura, K. P. and Victor, J. D." should be "N D Schiff and K P Purpura and J D Victor"
ReadEtal02: 	author "Readinger, Wilson O and Chatziastros, Astros and Cunningham, Douglas W and B{\"u}lthoff, Heinrich H and Cutting, James E" should be "Wilson O Readinger and Astros Chatziastros and Doug

ParkEtal96: 	author "Park, D C and Smith, A D and Lautenschlager, G and Earles, J L and Frieske, D and Zwahr, M and Gaines, C L" should be "D C Park and A D Smith and G Lautenschlager and J L Earles and D Frieske and M Zwahr and C L Gaines"
Fox89: 	author "Fox, S. E." should be "S E Fox"
DomnEtal13: 	author "Domnisoru, Cristina and Kinkhabwala, Amina A and Tank, David W" should be "Cristina Domnisoru and Amina A Kinkhabwala and David W Tank"
BrowEtal77: 	author "Brown, J. and Lewis, V. J. and Monk, A. F." should be "J Brown and V J Lewis and A F Monk"
Kell64: 	author "Kelley, H. P." should be "H P Kelley"
HoyeVerh06: 	author "Hoyer, W. J. and Verhaeghen, P." should be "W J Hoyer and P Verhaeghen"
VerhMarc93: 	author "Verhaeghen, P. and Marcoen, A." should be "P Verhaeghen and A Marcoen"
Scha87: 	author "Schacter, D. L." should be "D L Schacter"
CoheEich93: 	author "Cohen, N. J. and Eichenbaum, H." should be "N J Cohen and H Eichenbaum"
Ligh91: 	author "Light, L." should be "L Light"
Tu

Bron95: 	author "Bronzino, J. D." should be "J D Bronzino"
Koff35: 	author "Koffka, K." should be "K Koffka"
Crow76: 	author "Crowder, R. G." should be "R G Crowder"
KandEtal00: 	author "Kandel, E.R. and Schwartz, J.H. and Jessell, T.M." should be "E R Kandel and J H Schwartz and T M Jessell"
Chos05: 	author "Choset, H. M." should be "H M Choset"
NelsEtal89: 	author "Nelson, D. L. and Bajo, M. and McEvoy, C. L. and Schreiber, T. A." should be "D L Nelson and M Bajo and C L McEvoy and T A Schreiber"
Badd68: 	author "Baddeley, A. D." should be "A D Baddeley"
DaPo67: 	author "Da{P}olito, F. J." should be "F J Da{P}olito"
WickEtal63: 	author "Wickens, D. D. and Born, D. G. and Allen, C. K." should be "D D Wickens and D G Born and C K Allen"
CraiBirt71: 	author "Craik, F. I. M. and Birtwistle, J." should be "F I M Craik and J Birtwistle"
KeppUnde62: 	author "Keppel, G. and Underwood, B. J." should be "G Keppel and B J Underwood"
FozaWaug69: 	author "Fozard, J. J. and Waugh, N. C." should be

WilsEtal02: 	author "Wilson, H. R. and Loffler, G. and Wilkinson, F." should be "H R Wilson and G Loffler and F Wilkinson"
ShelMcNa01: 	author "Shelton, A. L. and McNamara, T. P." should be "A L Shelton and T P McNamara"
BaueEtal06: 	author "Bauer, M. and Oostenveld, R. and Peeters, M. and Fries, P." should be "M Bauer and R Oostenveld and M Peeters and P Fries"
CarsEtal99: 	author "Carstensen, L L and Isaacowitz, D M and Charles, S T" should be "L L Carstensen and D M Isaacowitz and S T Charles"
Mala12: 	author "Malach, Rafael" should be "Rafael Malach"
HayhEtal98: 	author "Hayhoe, M. M. and Bensinger, D. G. and Ballard, D. H." should be "M M Hayhoe and D G Bensinger and D H Ballard"
CaplEtal00: 	author "Caplan, J. B. and Kahana, M. J. and Sekuler, R. and Kirschen, M. P. and Madsen, J. R." should be "J B Caplan and M J Kahana and R Sekuler and M P Kirschen and J R Madsen"
PolyEtal06: 	author "Polyn, S. M. and Morton, N. W and Kogen, D. K. and Norman, K. A. and Kahana, M. J." should be

WillEtal05b: 	author "Williams, D. and Kuhn, A. and Kupsch, A. and Tijssen, M. and van Bruggen, G. and Speelman, H. and Hotton, G. and Loukas, C. and Brown, P." should be "D Williams and A Kuhn and A Kupsch and M Tijssen and G {van Bruggen} and H Speelman and G Hotton and C Loukas and P Brown"
HayaEtal08: 	author "Hayama, H.R. and Johnson, J.D. and Rugg, M.D." should be "H R Hayama and J D Johnson and M D Rugg"
McCaEtal10: 	author "Mc{C}abe, D. P. and Roediger, H. L. and McDaniel, M. A. and Balota, D. A. and Hambrick, D. Z." should be "D P Mc{C}abe and H L Roediger and M A McDaniel and D A Balota and D Z Hambrick"
FrisMiln90: 	author "V. Frisk and B. Milner" should be "V Frisk and B Milner"
Spen94: 	author "Spencer, S.S." should be "S S Spencer"
StonNysw27: 	author "Stone, C. P. and Nyswander, D. B." should be "C P Stone and D B Nyswander"
Sieg01: 	author "Siegel, J. M." should be "J M Siegel"
GlanEtal69: 	author "Glanzer, M. and Gianutsos, R. and Dubin, S" should be "M Glanzer and R G

Check editor names

In [16]:
fix_dict['editor'] = check_entries('editor', bd, [reformat_author(e) for e in editors])

running check: editor...
188 errors detected:
Kaha17: 	editor "J. H. Byrne" should be "J H Byrne"
ChenEtal15: 	editor "C. Cortes and N. D. Lawrence and D. D. Lee and M. Sugiyama and R. Garnett" should be "C Cortes and N D Lawrence and D D Lee and M Sugiyama and R Garnett"
Frie06: 	editor "C. Chen and W. H\"ardle and A Unwin" should be "C Chen and W H\"ardle and A Unwin"
Tulv07: 	editor "Gluck, M.A. and Anderson, J.R. and Kosslyn, S. M." should be "M A Gluck and JR Anderson and S M Kosslyn"
Heal14: 	editor "S. K. Whitbourne" should be "S K Whitbourne"
OReiEtal99: 	editor "Miyake, A. and Shah, P." should be "A Miyake and P Shah"
RehmNaus90: 	editor "R. E. Ingram" should be "R E Ingram"
Bowe67: 	editor "K. W. Spence and J. T. Spence" should be "K W Spence and J T Spence"
Mont98: 	editor "Egenhofer, Max J. and Golledge, Reginald G." should be "Max J Egenhofer and Reginald G Golledge"
BjorBjor92: 	editor "Healy, A. F. and Kosslyn, S. M." should be "A F Healy and S M Kosslyn"
RescWagn72: 	ed

Check addresses

In [17]:
fix_dict['address'] = check_entries('address', bd, [format_journal_name(a, key=address_key, force_caps=address_codes) for a in addresses])

running check: address...
292 errors detected:
NoceWrig06: 	address "New York" should be "New York, {NY}"
Frie06: 	address "Heidelberg" should be "Heidelberg, Germany"
TuftGrav83: 	address "Cheshire, CT" should be "Cheshire, {CT}"
BleiLaff06: 	address "New York, NY, US" should be "New York, {NY}"
Heal14: 	address "New York" should be "New York, {NY}"
Cowa97: 	address "New York, NY" should be "New York, {NY}"
OReiEtal99: 	address "New York" should be "New York, {NY}"
RehmNaus90: 	address "New York, NY" should be "New York, {NY}"
Bowe67: 	address "New York" should be "New York, {NY}"
Mont98: 	address "New York" should be "New York, {NY}"
BjorBjor92: 	address "Hillsdale, NJ, England" should be "Hillsdale, {NJ}"
RamaEtal12a: 	address "New Orleans, LA." should be "New Orleans, {LA}"
RescWagn72: 	address "New York" should be "New York, {NY}"
Mall98: 	address "San Diego" should be "San Diego, {CA}"
Mann11: 	address "Philadelphia, PA" should be "Philadelphia, {PA}"
Kapl76: 	address "Stroudsber

In [18]:
polished_bd = polish_database(bd, fix_dict, autofix=True)

removing extra fields...
processing BeatEtal16...
	removing field: date-modified
	removing field: date-added
processing LeeEtal20...
	removing field: date-modified
	removing field: date-added
processing ConsEtal16...
	removing field: date-modified
	removing field: date-added
processing BellEtal20...
	removing field: date-modified
	removing field: date-added
processing BellEtal18...
	removing field: date-modified
	removing field: date-added
processing GagnEtal20...
	removing field: date-modified
	removing field: date-added
processing TianEtal20...
	removing field: date-modified
	removing field: date-added
processing ArzyScha19...
	removing field: date-modified
	removing field: date-added
processing GautEtal18...
	removing field: date-modified
	removing field: date-added
processing GautvanW16...
	removing field: date-modified
	removing field: date-added
processing ArzyEtal09...
	removing field: date-modified
	removing field: date-added
processing PanzEtal10...
	removing field: date-modif

processing Radv12...
	removing field: date-modified
	removing field: date-added
processing ZwaaEtal95...
	removing field: date-modified
	removing field: date-added
processing ZackEtal01b...
	removing field: date-modified
	removing field: date-added
processing ChanEtal17...
	removing field: date-modified
	removing field: date-added
processing DuBrEtal17...
	removing field: date-modified
	removing field: date-added
processing AbraEtal14...
	removing field: date-modified
	removing field: date-added
processing FiscMode03...
	removing field: date-modified
	removing field: date-added
processing delaEtal16...
	removing field: date-modified
	removing field: date-added
processing RubiEtal17...
	removing field: date-modified
	removing field: date-added
processing ChanEtal12...
	removing field: date-modified
	removing field: date-added
processing Pold11...
	removing field: date-modified
	removing field: date-added
processing YarkEtal11...
	removing field: date-modified
	removing field: date-added

	removing field: isbn
	removing field: doi
	removing field: date-modified
	removing field: date-added
	removing field: abstract
processing MangEtal14...
	removing field: bdsk-url-2
	removing field: bdsk-url-1
	removing field: url
	removing field: pmid
	removing field: mendeley-groups
	removing field: issn
	removing field: isbn
	removing field: file
	removing field: doi
	removing field: date-modified
	removing field: date-added
	removing field: abstract
processing StatEtal15...
	removing field: bdsk-url-1
	removing field: pmid
	removing field: mendeley-groups
	removing field: issn
	removing field: isbn
	removing field: file
	removing field: doi
	removing field: date-modified
	removing field: date-added
	removing field: abstract
processing SnowEtal16...
	removing field: bdsk-url-1
	removing field: pmid
	removing field: mendeley-groups
	removing field: issn
	removing field: isbn
	removing field: file
	removing field: doi
	removing field: date-modified
	removing field: date-added
	removing

	removing field: doi
	removing field: date-modified
	removing field: date-added
	removing field: abstract
processing CurrEtal09...
	removing field: bdsk-url-1
	removing field: pmid
	removing field: keyword
	removing field: issn
	removing field: isbn
	removing field: file
	removing field: doi
	removing field: date-modified
	removing field: date-added
	removing field: abstract
processing DingEtal06...
	removing field: bdsk-url-1
	removing field: pmid
	removing field: keyword
	removing field: issn
	removing field: isbn
	removing field: file
	removing field: doi
	removing field: date-modified
	removing field: date-added
	removing field: abstract
processing DinoEtal16...
	removing field: bdsk-url-1
	removing field: pmid
	removing field: issn
	removing field: isbn
	removing field: file
	removing field: doi
	removing field: date-modified
	removing field: date-added
	removing field: abstract
processing EganEtal03...
	removing field: bdsk-url-1
	removing field: pmid
	removing field: issn
	remov

processing John94...
	removing field: date-modified
	removing field: date-added
processing KuncHezl07...
	removing field: date-modified
	removing field: date-added
processing Poph99...
	removing field: date-modified
	removing field: date-added
processing Jens80...
	removing field: date-modified
	removing field: date-added
processing HustEtal12...
	removing field: date-modified
	removing field: date-added
processing HillEtal05...
	removing field: date-modified
	removing field: date-added
processing BailEtal01...
	removing field: date-modified
	removing field: date-added
processing Snyd91...
	removing field: date-modified
	removing field: date-added
processing Sarv87...
	removing field: date-modified
	removing field: date-added
processing GuyoEtal02...
	removing field: date-modified
	removing field: date-added
processing Zar10...
	removing field: date-modified
	removing field: date-added
processing HutcEtal13...
	removing field: date-modified
	removing field: date-added
processing MacDEt

...

autocorrecting WorrEtal04[author] to "G A Worrell and L Parish and S D Cranstoun and R Jonas and G Baltuch and B Litt"
autocorrecting JirsEtal06[author] to "J D Jirsch and E Urrestarazu and P LeVan and A Olivier and F Dubeau and J Gotman"
autocorrecting GuggEtal07[author] to "A G Guggisberg and S S Dalal and A M Findlay and S S Nagarajan"
autocorrecting BragEtal99[author] to "A Bragin and J {Engel Jr} and C L Wilson and I Fried and G Buzs\'{a}ki"
autocorrecting ChroBuzs96[author] to "J J Chrobak and G Buzs\'aki"
autocorrecting LobbEtal11[author] to "C J Lobb and C J Wilson and C A Paladini"
autocorrecting KuhnEtal08[author] to "A A Kuhn and F Kempf and C Brucke and L {Gaynor Doyle} and I Martinez-Torres and A Pogosyan and T Trottenberg and A Kupsch and G H Schneider and M I Hariz and others"
autocorrecting LevyEtal00[author] to "R Levy and W D Hutchinson and A M Lozano and J O Dostrovsky"
autocorrecting GeviEtal97[author] to "A Gevins and M E Smith and L McEvoy and D Yu"
autocorrecting

autocorrecting VenaRipl02[author] to "W N Venables and B D Ripley"
autocorrecting BorgGroe05[author] to "I Borg and P J F Groenen"
autocorrecting JacoEtal05a[author] to "L L Jacoby and Y Shimizu and K A Daniels and M G Rhodes"
autocorrecting HikoWurt85[author] to "O Hikosaka and R H Wurtz"
autocorrecting LittEtal09[author] to "A S Little and K A Smith and K Kirlin and L C Baxter and S Chung and R Maganti and D M Treiman"
autocorrecting MonaAbbo11[author] to "J D Monaco and L F Abbott"
autocorrecting HermSpel96[author] to "L Hermer and E Spelke"
autocorrecting RuggEtal94[author] to "M D Rugg and M C Doyle and J S Holdstock"
autocorrecting RuggEtal92[author] to "M D Rugg and P Brovedani and M C Doyle"
autocorrecting JokiJens07[author] to "D Jokisch and O Jensen"
autocorrecting MullEtal00[author] to "M M M{\"u}ller and T Gruber and A Keil"
autocorrecting WomeEtal07[author] to "T Womelsdorf and J M Schoffelen and R Oostenveld and W Singer and R Desimone and A K Engel and P Fries"
autocorre

autocorrecting StarDava08[author] to "B P Staresina and L Davachi"
autocorrecting Kami69[author] to "L J Kamin"
autocorrecting McKiNoso96[author] to "S C McKinley and R M Nosofsky"
autocorrecting TiitEtal93[author] to "H Tiitinen and J Sinkkonen and K Reinikainen and K Alho and J Lavikainen and R Naatanen"
autocorrecting EngeMcCa10[author] to "A D Engell and G McCarthy"
autocorrecting SreeJha07[author] to "K K Sreenivasan and A P Jha"
autocorrecting BunsEich95[author] to "M Bunsey and H Eichenbaum"
autocorrecting CiarEtal07[author] to "E Ciaramelli and M Muccioli and E Ladavas and G {di Pellegrino}"
autocorrecting NasbYand82[author] to "William Nasby and Regina Yando"
autocorrecting WillEska06[author] to "Z M Williams and E N Eskandar"
autocorrecting MoorArms03[author] to "T Moore and K M Armstrong"
autocorrecting MorrEtal86[author] to "R Morris and E Anderson and G Lynch and M Baudry"
autocorrecting SchoEtal10[author] to "T Schonberg and J P O'Doherty and D Joel and R Inzelberg and Y 

autocorrecting KablGlim07[author] to "J W Kable and P W Glimcher"
autocorrecting LongKaha12b[author] to "N M Long and M J Kahana"
autocorrecting PaleEtal93[author] to "E Palesu and C D Frith and R S J Frackowiak"
autocorrecting ElliDola98[author] to "R Elliott and R J Dolan"
autocorrecting DEspEtal99[author] to "M D'{E}sposito and B R Postle and J Jonides and E E Smith"
autocorrecting McClEtal04b[author] to "S M McClure and M K York and P Read Montague"
autocorrecting HeekEtal08[author] to "H A Heekeren and S Marrett and L G Ungerleider"
autocorrecting KablGlim09[author] to "J W Kable and P W Glimcher"
autocorrecting AshbEll01[author] to "F G Ashby and S W Ell"
autocorrecting LlinEtal98[author] to "R Llinas and U Ribary and D Contreras and C Pedroarena"
autocorrecting MoscWino92[author] to "M Moscovitch and G Winocur"
autocorrecting Adey67b[author] to "W R Adey"
autocorrecting StelEtal08[author] to "B M L Stelten and L H M Noblesse and L Ackermans and Y Temel and V Visser-Vandewalle"
a

autocorrecting OReiEtal99[address] to "New York, {NY}"
autocorrecting RehmNaus90[address] to "New York, {NY}"
autocorrecting Bowe67[address] to "New York, {NY}"
autocorrecting Mont98[address] to "New York, {NY}"
autocorrecting BjorBjor92[address] to "Hillsdale, {NJ}"
autocorrecting RamaEtal12a[address] to "New Orleans, {LA}"
autocorrecting RescWagn72[address] to "New York, {NY}"
autocorrecting Mall98[address] to "San Diego, {CA}"
autocorrecting Mann11[address] to "Philadelphia, {PA}"
autocorrecting Kapl76[address] to "Stroudsberg, {PA}"
autocorrecting Crai77[address] to "New York, {NY}"
autocorrecting CraiRabi84[address] to "Hillsdale, {NJ}"
autocorrecting Crai00[address] to "Philadelphia, {PA}"
autocorrecting CraiByrd82[address] to "New York, {NY}"
autocorrecting LyonEtal06[address] to "Trieste, {IT}"
autocorrecting Este72[address] to "Washington, {DC}"
autocorrecting Jens62[address] to "Washington, {DC}"
autocorrecting SkagEtal93[address] to "San Mateo, {CA}"
autocorrecting Mitc96[ad

In [19]:
def writebib(fname, biblist, order, indent='\t'):
    def entry2str(e):
        s = '@' + e['ENTRYTYPE'] + '{' + e['ID'] + ','
        at_least_one = False
        for k in order:
            if (k not in ['ENTRYTYPE', 'ID']) and (k in e.keys()):
                s += '\n' + indent + k.capitalize() + ' = {' + e[k] + '},'
                at_least_one = True
        if at_least_one:
             s = s[:-1] #remove last comma

        return s + '}' + '\n'
    
    bibtex_str = '\n'.join([entry2str(b) for b in biblist])
    print(bibtex_str, file=open(fname, 'w'))

In [20]:
keep_fields = read('keep_fields.txt')
keep_fields.sort()

writebib('cleaned.bib', polished_bd, keep_fields)

To do:
- ~~handle first-author surnames that start with lowercase characters~~
- ~~better handling of curly braces to preserve name ordering~~
- ~~better handling of escape characters~~
- ~~ensure bibtex file compiles and gets parsed correctly~~
- ~~Check duplicate titles/authors (fail if detected)~~
- ~~Check bibtex keys.  Duplicates should be assigned a suffix of 'a', 'b', etc.~~
- ~~If keys match aside from suffix then still allow the bibtex file to "pass" as long as all "matching" keys are unique *and* all have suffixes *and* the suffixes span a, b, c, ..., etc. without gaps~~
~~- Correct page numberings:~~
~~  - Change hyphens to n-dashes~~
~~  - Change m-dashes to n-dashes~~
~~  - Remove spaces in page ranges~~
~~  - Print a warning for strangely formatted pages: leading 0, non-digits, very large numbers (greater than 10K?) but allow bibtex file to pass~~
- remove everything except for "keep_fields"
  - ~~Also add a "force" field that, when present, causes the checker to automatically pass that entry (if set to True); this will be used as a workaround for special cases not handled by the parser/checker~~
- ~~change all urls, dois, and page numbers to lowercase~~

Clean up:
- ~~correct all "strings" to full journal names~~
- ~~correct capitalization of journal names~~
- ~~correct all journal abbreviations to full journal names~~
- ~~Ensure no periods in journal names~~
- ~~Ensure no periods at the ends of affiliations or titles~~
- ~~Check for compressed initials (AA --> A A; A.A. --> A A; etc.)~~

Bonus:
- Use scholarly to verify information.  However, this is currently unreliable (server seems to hang frequently) and too slow to be viable